In [1]:
%load_ext dotenv
%dotenv

In [1]:
%%md
* EMMA - cex API websockets
* Replicate functionality REST and websocket
* Submit order REST and hear about it via websocket for L2 updates
* Orders that are placed, separate positions call

* L2 updates on websocket
* websocket feed for trade fills (personal only). DYDX pending, open, cancelled
* Submit over REST

UsageError: Cell magic `%%md` not found.


In [1]:
# TODO: test whirligig ws
# TODO: test with ccxt

import logging
import os
from zeta_py.zeta_client.accounts.pricing import Pricing
from solana.rpc.async_api import AsyncClient
from zeta_py.client import Client
from zeta_py.types import Network, Asset, Side
import anchorpy
from solders.pubkey import Pubkey
from zeta_py import pda
from solana.rpc.commitment import Confirmed, Finalized
from datetime import datetime, timedelta
from zeta_py.types import OrderOptions, OrderType
from solana.rpc.types import TxOpts
from datetime import datetime, timedelta
from zeta_py.types import OrderOptions, OrderType
from zeta_py import utils

logger = logging.getLogger("zeta_py")
logging.basicConfig(level=logging.DEBUG)

endpoint = os.getenv("ENDPOINT", "https://api.mainnet-beta.solana.com")
print(endpoint.split(".com")[0])
conn = AsyncClient(endpoint, commitment=Confirmed, blockhash_cache=False)

wallet = anchorpy.Wallet.local()  # get local filesystem keypair wallet
print(wallet.public_key)

https://api.mainnet-beta.solana
4mYxEXiFwe3ePEmoj7CYCbiW7wGTojUQCmdPqGaoVaUh


In [2]:
print(f"Balance: {(await conn.get_balance(wallet.public_key)).value / 1e9} SOL")

Balance: 0.21848177 SOL


In [3]:
tx_opts = TxOpts({"preflight_commitment": Confirmed, "skip_confirmation": False})

client = await Client.load(Network.MAINNET, conn, wallet, [Asset.SOL], tx_opts=tx_opts)

In [ ]:
client

In [8]:
print(client.balance)
print(client.positions)
print(client.open_orders)

AttributeError: 'Client' object has no attribute 'balance'

In [44]:
await client.deposit(0.1)

INFO:zeta_py.accounts.Account:Loaded account: CrossMarginAccountManager
INFO:zeta_py.client.Client:Depositing 0.1 USDC to margin account


Signature(
    YXELCRwZZyZDFMRSbmB4AFf5JiHDARhxzSios4RboXsPximyP8BbtcbMno5mELL5eTDfm81WPc9tMu7Gn386EgY,
)

In [ ]:
# Place order

expiry_ts = int((datetime.now() + timedelta(seconds=1000)).timestamp())
order_opts = OrderOptions(expiry_ts=expiry_ts, order_type=OrderType.PostOnlySlide, client_order_id=69)
await client.place_order(Asset.SOL, 0.1, 0.001, Side.Ask, order_opts)

# Quoting
Use this code to atomically cancel and replace a set of quotes on Zeta.

*For this example we quote 0.001 SOL-PERP contracts at 20bps spread*

In [7]:
# Quoting parameters
asset = Asset.SOL  # SOL-PERP market
quote_size = 0.001  # min size
quote_bps = 10  # basis points from midpoint
time_in_force = 10  # expire the quote after 10 seconds

# Get the latest mark price and calculate bid/ask prices
price_fixed = client.exchange.pricing.account.mark_prices[asset.to_index()]
bid_price = utils.convert_fixed_int_to_decimal(price_fixed * (1 - quote_bps / 10000))
ask_price = utils.convert_fixed_int_to_decimal(price_fixed * (1 + quote_bps / 10000))

# Set order options
expiry_ts = int((datetime.now() + timedelta(seconds=time_in_force)).timestamp())
order_opts = OrderOptions(expiry_ts=expiry_ts, order_type=OrderType.Limit, client_order_id=69)

# Execute quote!
await client.replace_quote(Asset.SOL, bid_price, quote_size, ask_price, quote_size, order_opts)

INFO:zeta_py.client.Client:Replacing SOL orders: 0.001x Bid @ $21.757750469999998, 0.001x Ask @ $21.801309529999997


Signature(
    3Lb5vqF3pQKSqXBLm4hx4EMgRtMJctcSuwgZ8unRfXF9wgFewQuXF7JMTpgDDURQMcPMfMLy1XifVD9P6o6KgMDd,
)

In [8]:
open_orders = await client.fetch_open_orders(Asset.SOL)
print("Open Orders:")
for order in open_orders:
    print(f"{order.side.name} {order.info.size}x ${order.info.price}")

Open Orders:
Bid 0.001x $21.7577
Ask 0.001x $21.8013


ERROR:zeta_py.accounts.Account:Error subscribing to Clock: no close frame received or sent
ERROR:zeta_py.accounts.Account:Error subscribing to Pricing: no close frame received or sent


In [ ]:
oid = open_orders[0].order_id
await client.cancel_order(Asset.SOL, order_id=oid, side=Side.Ask)

In [45]:
import asyncio
import ccxt.async_support as ccxt

async def quote(client, asset: Asset, price):
    # Quoting parameters
    quote_size = 0.001  # min size
    quote_bps = 5  # basis points from midpoint
    time_in_force = 30  # expire the quote after 10 seconds

    # Get the latest mark price and calculate bid/ask prices
    # price_fixed = client.exchange.pricing.account.mark_prices[asset.to_index()]
    bid_price = price * (1 - quote_bps / 10000)
    ask_price = price * (1 + quote_bps / 10000)

    # Set order options
    expiry_ts = int((datetime.now() + timedelta(seconds=time_in_force)).timestamp())
    order_opts = OrderOptions(expiry_ts=expiry_ts, order_type=OrderType.PostOnlySlide, client_order_id=69)

    # Execute quote!
    await client.replace_quote(asset, bid_price, quote_size, ask_price, quote_size, order_opts)
    
async def get_binance_midpoint(exchange, symbol):
    orderbook = await exchange.fetch_order_book(symbol, limit=1)
    return (orderbook['bids'][0][0]+orderbook['asks'][0][0])/2

async def fetch_ticker_continuously(symbol):
    # you can set enableRateLimit = True to enable the built-in rate limiter
    # this way you request rate will never hit the limit of an exchange
    # the library will throttle your requests to avoid that

    exchange = ccxt.binance()
    # exchange.load_markets()
    mid_price = await get_binance_midpoint(exchange, symbol)
    await quote(client, Asset.SOL, mid_price)
    while True:
        print('--------------------------------------------------------------')
        print(exchange.iso8601(exchange.milliseconds()), 'fetching', symbol, 'ticker from', exchange.name)
        # this can be any call instead of fetch_ticker, really
        try:
            # ticker = await exchange.fetch_ticker(symbol)
            # orderbook = await exchange.fetch_order_book(symbol, limit=1)
            # print(exchange.iso8601(exchange.milliseconds()), 'fetched', symbol, 'ticker from', exchange.name)
            new_mid_price = await get_binance_midpoint(exchange, symbol)
            print(new_mid_price)
            if new_mid_price != mid_price:
                mid_price = new_mid_price
                print(f'Binance price moved to {mid_price}, requoting on Zeta...')
                await quote(client, Asset.SOL, mid_price)
            
        except ccxt.RequestTimeout as e:
            print('[' + type(e).__name__ + ']')
            print(str(e)[0:200])
            # will retry
        except ccxt.DDoSProtection as e:
            print('[' + type(e).__name__ + ']')
            print(str(e.args)[0:200])
            # will retry
        except ccxt.ExchangeNotAvailable as e:
            print('[' + type(e).__name__ + ']')
            print(str(e.args)[0:200])
            # will retry
        except ccxt.ExchangeError as e:
            print('[' + type(e).__name__ + ']')
            print(str(e)[0:200])
            break  # won't retry
          
symbol = "SOL/USDT"

await fetch_ticker_continuously(symbol)

INFO:zeta_py.client.Client:Replacing SOL orders:                 0.001x Bid @ $19.2353775, 0.001x Ask @ $19.254622499999996


--------------------------------------------------------------
2023-09-05T04:03:13.827Z fetching SOL/USDT ticker from Binance
19.244999999999997
--------------------------------------------------------------
2023-09-05T04:03:13.965Z fetching SOL/USDT ticker from Binance
19.244999999999997
--------------------------------------------------------------
2023-09-05T04:03:14.100Z fetching SOL/USDT ticker from Binance
19.244999999999997
--------------------------------------------------------------
2023-09-05T04:03:14.235Z fetching SOL/USDT ticker from Binance
19.244999999999997
--------------------------------------------------------------
2023-09-05T04:03:14.379Z fetching SOL/USDT ticker from Binance
19.244999999999997
--------------------------------------------------------------
2023-09-05T04:03:14.521Z fetching SOL/USDT ticker from Binance
19.244999999999997
--------------------------------------------------------------
2023-09-05T04:03:14.663Z fetching SOL/USDT ticker from Binance
19.2

INFO:zeta_py.client.Client:Replacing SOL orders:                 0.001x Bid @ $19.225382500000002, 0.001x Ask @ $19.244617499999997


19.244999999999997
--------------------------------------------------------------
2023-09-05T04:03:33.583Z fetching SOL/USDT ticker from Binance
19.235
Binance price moved to 19.235, requoting on Zeta...
--------------------------------------------------------------
2023-09-05T04:03:34.135Z fetching SOL/USDT ticker from Binance
19.235
--------------------------------------------------------------
2023-09-05T04:03:34.277Z fetching SOL/USDT ticker from Binance
19.235
--------------------------------------------------------------
2023-09-05T04:03:34.410Z fetching SOL/USDT ticker from Binance
19.235
--------------------------------------------------------------
2023-09-05T04:03:34.547Z fetching SOL/USDT ticker from Binance
19.235
--------------------------------------------------------------
2023-09-05T04:03:34.682Z fetching SOL/USDT ticker from Binance
19.235
--------------------------------------------------------------
2023-09-05T04:03:34.824Z fetching SOL/USDT ticker from Binance
19.23

INFO:zeta_py.client.Client:Replacing SOL orders:                 0.001x Bid @ $19.215387500000002, 0.001x Ask @ $19.2346125


...

INFO:zeta_py.client.Client:Replacing SOL orders:                 0.001x Bid @ $19.2353775, 0.001x Ask @ $19.254622499999996


19.244999999999997
Binance price moved to 19.244999999999997, requoting on Zeta...
--------------------------------------------------------------
2023-09-05T04:10:20.753Z fetching SOL/USDT ticker from Binance
19.244999999999997
--------------------------------------------------------------
2023-09-05T04:10:20.888Z fetching SOL/USDT ticker from Binance
19.244999999999997
--------------------------------------------------------------
2023-09-05T04:10:21.024Z fetching SOL/USDT ticker from Binance
19.244999999999997
--------------------------------------------------------------
2023-09-05T04:10:21.161Z fetching SOL/USDT ticker from Binance
19.244999999999997
--------------------------------------------------------------
2023-09-05T04:10:21.297Z fetching SOL/USDT ticker from Binance
19.244999999999997
--------------------------------------------------------------
2023-09-05T04:10:21.442Z fetching SOL/USDT ticker from Binance
19.244999999999997
-----------------------------------------------

INFO:zeta_py.client.Client:Replacing SOL orders:                 0.001x Bid @ $19.225382500000002, 0.001x Ask @ $19.244617499999997


19.244999999999997
--------------------------------------------------------------
2023-09-05T04:10:47.737Z fetching SOL/USDT ticker from Binance
19.235
Binance price moved to 19.235, requoting on Zeta...
--------------------------------------------------------------
2023-09-05T04:10:48.319Z fetching SOL/USDT ticker from Binance
19.235
--------------------------------------------------------------
2023-09-05T04:10:48.453Z fetching SOL/USDT ticker from Binance
19.235
--------------------------------------------------------------
2023-09-05T04:10:48.602Z fetching SOL/USDT ticker from Binance
19.235
--------------------------------------------------------------
2023-09-05T04:10:48.728Z fetching SOL/USDT ticker from Binance
19.235
--------------------------------------------------------------
2023-09-05T04:10:48.874Z fetching SOL/USDT ticker from Binance
19.235
--------------------------------------------------------------
2023-09-05T04:10:49.009Z fetching SOL/USDT ticker from Binance
19.23

CancelledError: 

In [29]:
from binance.cm_futures import CMFutures

# WebSocket Stream Client
import time
from binance.websocket.um_futures.websocket_client import UMFuturesWebsocketClient

def message_handler(_, message):
    logging.info(message)

my_client = UMFuturesWebsocketClient(on_message=message_handler)

# Subscribe to a single symbol stream
my_client.book_ticker(symbol="solusdt")
time.sleep(10)
logging.info("closing ws connection")
my_client.stop()

INFO:root:{"result":null,"id":1693879865955}
INFO:root:{"e":"bookTicker","u":3234204813050,"s":"SOLUSDT","b":"19.2860","B":"476","a":"19.2870","A":"109","T":1693879866865,"E":1693879866870}
INFO:root:{"e":"bookTicker","u":3234204821663,"s":"SOLUSDT","b":"19.2860","B":"478","a":"19.2870","A":"109","T":1693879867219,"E":1693879867222}
INFO:root:{"e":"bookTicker","u":3234204825737,"s":"SOLUSDT","b":"19.2860","B":"478","a":"19.2870","A":"106","T":1693879867359,"E":1693879867363}
INFO:root:{"e":"bookTicker","u":3234204826038,"s":"SOLUSDT","b":"19.2860","B":"478","a":"19.2870","A":"110","T":1693879867372,"E":1693879867378}
INFO:root:{"e":"bookTicker","u":3234204836813,"s":"SOLUSDT","b":"19.2870","B":"344","a":"19.2880","A":"67","T":1693879867674,"E":1693879867680}
INFO:root:{"e":"bookTicker","u":3234204836924,"s":"SOLUSDT","b":"19.2870","B":"407","a":"19.2880","A":"67","T":1693879867674,"E":1693879867680}
INFO:root:{"e":"bookTicker","u":3234204836979,"s":"SOLUSDT","b":"19.2870","B":"475","a"

In [10]:
import json
from binance import AsyncClient, DepthCacheManager, BinanceSocketManager

async def main():

    # initialise the client
    client = await AsyncClient.create()

    # run some simple requests
    # print(json.dumps(await client.get_exchange_info(), indent=2))

    # print(json.dumps(await client.get_symbol_ticker(symbol="SOLUSDT"), indent=2))

    # initialise websocket factory manager
    bsm = BinanceSocketManager(client)
    
    # setup an async context the Depth Cache and exit after 5 messages
    async with DepthCacheManager(client, symbol='SOLUSDT', ws_interval=0, limit=1) as dcm_socket:
        for _ in range(50000):
            depth_cache = await dcm_socket.recv()
            print(f"symbol {depth_cache.symbol} updated:{depth_cache.update_time}")
            print(f"Best bid: {depth_cache.get_bids()[:1]}, Best ask: {depth_cache.get_asks()[:1]}")
            
await main()

symbol SOLUSDT updated:1693880667775
Best bid: [[19.28, 821.53]], Best ask: [[19.29, 3062.97]]
symbol SOLUSDT updated:1693880668775
Best bid: [[19.28, 821.53]], Best ask: [[19.29, 3062.97]]
symbol SOLUSDT updated:1693880669775
Best bid: [[19.28, 899.36]], Best ask: [[19.29, 3062.97]]
symbol SOLUSDT updated:1693880670775
Best bid: [[19.28, 914.78]], Best ask: [[19.29, 3069.43]]
symbol SOLUSDT updated:1693880671775
Best bid: [[19.28, 914.78]], Best ask: [[19.29, 3069.43]]
symbol SOLUSDT updated:1693880672775
Best bid: [[19.28, 914.78]], Best ask: [[19.29, 3069.43]]
symbol SOLUSDT updated:1693880673775
Best bid: [[19.28, 914.78]], Best ask: [[19.29, 3069.43]]
symbol SOLUSDT updated:1693880674775
Best bid: [[19.28, 836.95]], Best ask: [[19.29, 3257.23]]
symbol SOLUSDT updated:1693880675775
Best bid: [[19.28, 821.53]], Best ask: [[19.29, 3257.23]]
symbol SOLUSDT updated:1693880676775
Best bid: [[19.28, 821.53]], Best ask: [[19.29, 3257.23]]
symbol SOLUSDT updated:1693880677775
Best bid: [[1

CancelledError: 

In [24]:
import re
from solana.rpc.websocket_api import connect

address = client.exchange.pricing.address
ws_endpoint = re.sub(r"^http", "ws", endpoint)
async with connect(ws_endpoint) as ws:
  await ws.account_subscribe(
      address,
      commitment=client.exchange.connection.commitment,
      encoding="base64+zstd",
  )
  first_resp = await ws.recv()
  first_resp[0].result
  async for msg in ws:
      account = client.exchange.pricing.decode(msg[0].result.value.data)
      print([utils.convert_fixed_int_to_decimal(p) for p in account.mark_prices])

[19.746566, 25734.475, 1630.92, 5.47435, 0.899192]
[19.746014, 25734.462861, 1630.91, 5.474999, 0.899192]
[19.746014, 25734.462861, 1630.91, 5.474999, 0.89916]
[19.74715, 25734.595, 1630.915, 5.474556, 0.89916]
[19.748712, 25735.0, 1630.91, 5.47435, 0.899202]
[19.75, 25735.0, 1630.915, 5.474999, 0.89916]
[19.749911, 25735.0, 1630.915, 5.474999, 0.89916]
[19.749911, 25735.0, 1630.95, 5.474999, 0.89916]
[19.749911, 25735.0, 1630.915, 5.474999, 0.89916]
[19.749792, 25734.551546, 1630.92, 5.47445, 0.899127]
[19.75, 25734.551949, 1630.92, 5.474901, 0.899127]
[19.75, 25734.551949, 1630.95, 5.474999, 0.89916]
[19.749412, 25734.560939, 1630.9, 5.47445, 0.89916]
[19.75, 25734.560939, 1630.91, 5.4739, 0.89916]
[19.748609, 25734.548841, 1630.9, 5.474999, 0.89916]
[19.749792, 25734.543278, 1630.9, 5.474999, 0.89916]
[19.749154, 25734.543278, 1630.9, 5.474999, 0.89916]
[19.747762, 25733.027242, 1630.9, 5.474999, 0.89916]
[19.747683, 25733.0, 1630.9, 5.474999, 0.89916]
[19.748262, 25734.05, 1630.9, 

CancelledError: 

In [7]:
def pricing_callback(account: Pricing):
    print([utils.convert_fixed_int_to_decimal(p) for p in account.mark_prices])
    
client.exchange.pricing.subscribe(Network.MAINNET, Confirmed, pricing_callback)

INFO:zeta_py.accounts.Account:Subscribed to Pricing


[19.470266, 25744.445, 1632.92602, 5.493726, 0.900162]
[19.470266, 25745.0, 1632.915344, 5.4939, 0.900159]
[19.473, 25745.0, 1632.965318, 5.49454, 0.900214]
[19.4762, 25746.5, 1633.022402, 5.494838, 0.900342]
[19.475, 25746.75, 1632.967311, 5.495076, 0.900349]
[19.475, 25746.36, 1632.982311, 5.494976, 0.900349]
[19.473895, 25747.0, 1632.982311, 5.4951, 0.900349]


In [8]:
client.exchange.pricing.unsubscribe()

INFO:zeta_py.accounts.Account:Unsubscribed to Pricing


INFO:zeta_py.accounts.Account:WebSocket subscription task cancelled.


In [8]:
client.exchange.markets[Asset.SOL].subscribe_orderbooks()

INFO:zeta_py.market.Market.SOL:Subscribed to SOL:bid
INFO:zeta_py.market.Market.SOL:Subscribed to SOL:ask


In [21]:
import re
from anchorpy import EventParser, Program
from solana.rpc.websocket_api import connect
from solders.rpc.config import RpcTransactionLogsFilterMentions

address = client.exchange.pricing.address
ws_endpoint = re.sub(r"^http", "ws", endpoint)
async with connect(ws_endpoint) as ws:
    await ws.logs_subscribe(
        commitment=client.exchange.connection.commitment,
        filter_=RpcTransactionLogsFilterMentions(Pubkey.from_string("ZETAxsqBRek56DhiGXrn75yj2NHU3aYUnxvHXpkf3aD"))
    )
    first_resp = await ws.recv()
    first_resp[0].result
    while True:
        msg = await ws.recv()
        logs = msg[0].result.value.logs
        parser = EventParser(client.exchange.program_id, client.exchange.program.coder)
        parsed = []
        parser.parse_logs(logs, lambda evt: parsed.append(evt))
        for event in parsed:
            if event.name == "ApplyFundingEvent":
                pass
            elif event.name == "PlaceOrderEvent":
                # if event.data.margin_account == client.margin_account.address:
                #     print(event)
                pass
            elif event.name == "OrderCompleteEvent":
                if event.data.margin_account == client.margin_account.address:
                    print(event)
            elif event.name == "TradeEventV3":
                if event.data.margin_account == client.margin_account.address:
                    print(event)
            elif event.name == "LiquidationEvent":
                if event.data.liquidatee == client
            else:
                print(event)

Event(name='TradeEventV3', data=TradeEventV3(margin_account=Pubkey(
    Hu2yXbPESXyamLdvDemeASCFd3rHnpN2xBDUJpnuG5UX,
), index=137, size=1, cost_of_trades=19646, is_bid=True, client_order_id=0, order_id=369591604009959165955688551, asset=Asset.SOL(), user=Pubkey(
    4mYxEXiFwe3ePEmoj7CYCbiW7wGTojUQCmdPqGaoVaUh,
), is_taker=True, sequence_number=168634592, fee=19))


CancelledError: 

ERROR:zeta_py.accounts.Account:Error subscribing to Clock: no close frame received or sent
ERROR:zeta_py.accounts.Account:Error subscribing to Clock: no close frame received or sent
ERROR:zeta_py.accounts.Account:Error subscribing to Clock: no close frame received or sent
ERROR:zeta_py.accounts.Account:Error subscribing to Clock: no close frame received or sent
ERROR:zeta_py.accounts.Account:Error subscribing to Clock: no close frame received or sent
ERROR:zeta_py.accounts.Account:Error subscribing to Clock: no close frame received or sent
ERROR:zeta_py.accounts.Account:Error subscribing to Clock: no close frame received or sent
ERROR:zeta_py.accounts.Account:Error subscribing to Clock: no close frame received or sent
ERROR:zeta_py.accounts.Account:Error subscribing to Clock: no close frame received or sent
ERROR:zeta_py.accounts.Account:Error subscribing to Clock: no close frame received or sent
ERROR:zeta_py.accounts.Account:Error subscribing to Clock: no close frame received or sent